In [1]:
import os
import inspect
os.environ["XLA_PYTHON_CLIENT_PREALLOCATE"] = "false"
from tracr.rasp import rasp
from tracr.compiler import compiling, rasp_to_graph
import numpy as np
from tracr.compiler.validating import validate
from typing import Union, TypeVar
from rasp_generator import map_primitives, sampling, utils
import networkx as nx

from tracr.rasp.rasp import Map, SequenceMap, LinearSequenceMap, Select, Aggregate, Comparison, SelectorWidth, indices, tokens 


def compile_rasp_to_model(sop: rasp.SOp, vocab={1,2,3,4}, max_seq_len=5, compiler_bos="BOS"):
    return compiling.compile_rasp_to_model(
        sop,
        vocab=vocab,
        max_seq_len=max_seq_len,
        compiler_bos=compiler_bos
    )


TEST_INPUT = [1,2,3,4]

# Program Generation

In [2]:
# Remaining problems
# - sometimes categorical Aggregate is hard to sample (reaches max retries)
# - sometimes a sampled program doesn't depend on rasp.tokens
# - sometimes programs are trivial (e.g. output is all Nones)
# - I suspect that sometimes the output is constant. TODO: check multiple inputs


# TODOS
# - remove SOps that are all (or mostly) None
# - maybe remove / downweight constant SOps?

In [36]:
sampler = sampling.ProgramSampler(validate_compilation=True)
sampler.sample()
print(sampler.output([1,2,3,4]))

ValueError: Compiled program aggregate_51 does not match RASP output.
Compiled output: [1, 1, 2, 3]
RASP output: [None, 1, 2, 3]
Test input: [1, 2, 3, 4]
SOp: <tracr.rasp.rasp.Aggregate object at 0x7f734cc2bb20>

In [37]:
utils.print_program(sampler.sops[2])

select_52 = Select(tokens, indices, predicate=Comparison.EQ)
aggregate_51 = Aggregate(select_52, tokens)    # type: categorical


In [14]:
# print program with and without test input
k = 1
utils.print_program(sampler.sops[-k])
print()
print()
utils.print_program(sampler.sops[-k], TEST_INPUT)

select_15 = Select(indices, indices, predicate=Comparison.EQ)
select_23 = Select(indices, indices, predicate=Comparison.EQ)
select_24 = Select(indices, tokens, predicate=Comparison.NEQ)
aggregate_14 = Aggregate(select_15, indices)    # type: categorical
selector_width_22 = SelectorWidth(select_24)    # type: categorical
aggregate_21 = Aggregate(select_23, selector_width_22)    # type: categorical
select_20 = Select(aggregate_21, selector_width_22, predicate=Comparison.GT)
selector_width_19 = SelectorWidth(select_20)    # type: categorical
select_18 = Select(aggregate_14, selector_width_19, predicate=Comparison.LT)
aggregate_17 = Aggregate(select_18, selector_width_19)    # type: categorical
sequence_map_16 = SequenceMap(lambda x, y: x*y, aggregate_17, indices)    # type: categorical


select_15 = Select(indices, indices, predicate=Comparison.EQ)
select_23 = Select(indices, indices, predicate=Comparison.EQ)
select_24 = Select(indices, tokens, predicate=Comparison.NEQ)
aggregate_14 = Agg